1、原始最高50左右，结合ESIM后57，加入pretrain的embedding效果变差

In [36]:
import sys
import jieba
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import gensim
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from torchtext import data
from time import time
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# hyper parameters
STR_MAXLEN = 30
BATCH_SIZE = 256
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
EMBED_DIM = 300
HIDDEN_DIM = 150
DEEP_LAYERS = [300]
LEARNING_RATE = 1e-3*0.95
EPOCHES = 20
DECAY_STEP = 2
DECAY_GAMMA = 0.99
CLASS_WEIGHT = [0.6116, 2.7397]
def print_flush(data, args=None):
    if args == None:
        print(data)
    else:
        print(data, args)
    sys.stdout.flush()
    
def pad_seq(seq, max_length):
    length = len(seq[0])
    pad_leng = 0 if length > max_length else (max_length-length)
    if pad_leng == 0:
        seq = seq[:, :max_length]
    else:
        seq = torch.cat([seq, torch.ones(len(seq), pad_leng).long().to(DEVICE)], dim=1)
    return seq

def Frobenius(mat):
    size = mat.size()
    if len(size) == 3:  # batched matrix
        ret = (torch.sum(torch.sum((mat ** 2), 2), 1).squeeze() + 1e-10) ** 0.5
        return torch.sum(ret) / size[0]
    else:
        raise Exception('matrix for computing Frobenius norm should be with 3 dims')

def get_mask(txt1, txt2):
    txt1 = txt1.cpu().numpy()
    txt2 = txt2.cpu().numpy()
    len_txt1 = len(txt1[0])
    len_txt2 = len(txt2[0])
    mask_txt1 = np.zeros(shape=[len(txt1), len_txt1])
    mask_txt2 = np.zeros(shape=[len(txt2), len_txt2])
    for i in range(len(txt1)):
        mask_len1 = np.where(txt1[i] == 1)[0]
        mask_len2 = np.where(txt2[i] == 1)[0]
        if len(mask_len1) == 0:
            mask_txt1[i, :] = 1
        else:
            mask_txt1[i, :mask_len1[0]] = 1
        if len(mask_len2) == 0:
            mask_txt2[i, :] = 1
        else:
            mask_txt2[i, :mask_len2[0]] = 1
    return torch.tensor(mask_txt1).float().to(DEVICE), torch.tensor(mask_txt2).float().to(DEVICE)       

def wordmodel_to_matrix(pretrain_path, wordlist, device, dim=300):
    word_vec = gensim.models.Word2Vec.load(pretrain_path).wv
    word_vec_list = []
    oov = 0
    oov_words = []
    for idx, word in enumerate(wordlist):
        try:
            vector = np.array(word_vec[word], dtype=float).reshape(1,dim)
        except:
            oov += 1
            oov_words.append(word)
            # print(word)
            vector = np.random.rand(1, dim)
        word_vec_list.append(torch.from_numpy(vector))
    wordvec_matrix = torch.cat(word_vec_list)
    print("Load embedding finished.")
    print("Total words count: {}, oov count: {}.".format(wordvec_matrix.size()[0], oov))
    return wordvec_matrix if device == -1 else wordvec_matrix.to(device)

def wordlist_to_matrix(pretrain_path, wordlist, device, dim=300):
    word_vec = {}
    with open(pretrain_path, encoding='utf-8') as fr:
        for line in fr:
            line = line.split(' ')
            word = line[0]
            vec = line[1:]
            word_vec[word] = np.array(vec, dtype=float)
    word_vec_list = []
    oov = 0
    oov_words = []
    for idx, word in enumerate(wordlist):
        try:
            vector = np.array(word_vec[word], dtype=float).reshape(1,dim)
        except:
            oov += 1
            oov_words.append(word)
            # print(word)
            vector = np.random.rand(1, dim)
        word_vec_list.append(torch.from_numpy(vector))
    wordvec_matrix = torch.cat(word_vec_list)
    print("Load embedding finished.")
    print("Total words count: {}, oov count: {}.".format(wordvec_matrix.size()[0], oov))
    return wordvec_matrix if device == -1 else wordvec_matrix.to(device)

torch.manual_seed(666)

In [37]:
# prepare data

def tokenizer(text): # create a tokenizer function
    return [txt for txt in text]
class BatchWrapper:
    def __init__(self, dl, iter_columns):
        self.dl, self.iter_columns = dl, iter_columns  # we pass in the list of attributes for x &amp;amp;amp;amp;lt;g class="gr_ gr_3178 gr-alert gr_spell gr_inline_cards gr_disable_anim_appear ContextualSpelling ins-del" id="3178" data-gr-id="3178"&amp;amp;amp;amp;gt;and y&amp;amp;amp;amp;lt;/g&amp;amp;amp;amp;gt;

    def __iter__(self):
        for batch in self.dl:
            yield (getattr(batch, attr) for attr in self.iter_columns)

    def __len__(self):
        return len(self.dl)
    
print_flush('process raw data...')
TEXT = data.Field(sequential=True, use_vocab=True, eos_token='<EOS>', init_token='<BOS>',pad_token='<PAD>', 
                  batch_first=True, tokenize=tokenizer)
LABEL = data.Field(sequential=False, use_vocab=False, batch_first=True)

tv_datafields = [("id", None), # we won't be needing the id, so we pass in None as the field
                 ("txt1", TEXT), ("txt2", TEXT),
                 ("label", LABEL)]

train = data.TabularDataset(path='../datasets/train_random.csv', format='csv', skip_header=True, fields=tv_datafields)
valid = data.TabularDataset(path='../datasets/valid_random.csv', format='csv', skip_header=True, fields=tv_datafields)

TEXT.build_vocab(train, valid, min_freq=3)
print_flush('Building vocabulary Finished.')

matrix = wordmodel_to_matrix('../datasets/pretrain_embedding/raw_embedding_300d.bin', TEXT.vocab.itos, DEVICE)

train_iter = data.BucketIterator(dataset=train, batch_size=BATCH_SIZE, sort_key=lambda x: len(x.txt1) + len(x.txt2), shuffle=True, device=-1, repeat=False)
valid_iter = data.Iterator(dataset=valid, batch_size=BATCH_SIZE, device=-1, shuffle=False, repeat=False)

train_dl = BatchWrapper(train_iter, ["txt1", "txt2", "label"])
valid_dl = BatchWrapper(valid_iter, ["txt1", "txt2", "label"])

print_flush('prepare data done!')

process raw data...
Building vocabulary Finished.
Load embedding finished.
Total words count: 1517, oov count: 216.


In [40]:
class Hybrid_cnn_rnn(nn.Module):
    def __init__(self, vocab, embed_dim, hidden_dim, deep_layers, pretrain_embed=torch.tensor([]), is_batch_norm=False, is_drop_out=False):
        super(Hybrid_cnn_rnn, self).__init__()
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.deep_layers = deep_layers
        self.is_batch_norm = is_batch_norm
        self.is_drop_out = is_drop_out
        self.word_embed = nn.Embedding(len(vocab), embed_dim, padding_idx=vocab['<PAD>'])
        if len(pretrain_embed) != 0:
            self.word_embed.weight.data.copy_(pretrain_embed)
        self.lstm_embed = nn.LSTM(embed_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.cnn_extract = nn.Conv2d(1, 5, (hidden_dim*2, 1))
        self.linear_1 = nn.Linear(hidden_dim*2, deep_layers[0])
        if self.is_batch_norm:
            self.batch_norm_1 = nn.BatchNorm1d(deep_layers[0])
        if self.is_drop_out:
            self.linear_1_dropout = nn.Dropout(0.8)
        for i, h in enumerate(self.deep_layers[1:], 1):
            setattr(self,'linear_'+str(i+1), nn.Linear(self.deep_layers[i-1], self.deep_layers[i]))
            if self.is_batch_norm:
                setattr(self, 'batch_norm_' + str(i + 1), nn.BatchNorm1d(deep_layers[i]))
            if self.is_drop_out:
                setattr(self, 'linear_'+str(i+1)+'_dropout', nn.Dropout(0.8))
#         self.attention_feature_layer = nn.Linear(STR_MAXLEN, hidden_dim*2)
        self.lstm_fusion = nn.LSTM(2*hidden_dim*4+5, hidden_dim, batch_first=True, bidirectional=True)
        self.linear_1 = nn.Linear(2*hidden_dim*2, deep_layers[0])
        self.deep_out = nn.Linear(deep_layers[-1], 2)
#         self.deep_dropout_1 = nn.Dropout(0.5)
    def forward(self, txt1, txt2, x1_mask, x2_mask, hidden=None):
        embed_txt1 = self.word_embed(txt1)
        embed_txt2 = self.word_embed(txt2)
        lstm_txt1 = self.lstm_embed(embed_txt1, None)[0]
        lstm_txt2 = self.lstm_embed(embed_txt2, None)[0]
        lstm_txt1 = lstm_txt1 * x1_mask[:, :, None]
        lstm_txt2 = lstm_txt2 * x2_mask[:, :, None]
        lstm_txt1_ = lstm_txt1.transpose(2, 1).unsqueeze(1)
        lstm_txt2_ = lstm_txt2.transpose(2, 1).unsqueeze(1)
        cnn_encode1 = F.relu(self.cnn_extract(lstm_txt1_)).squeeze().transpose(1, 2)
        cnn_encode2 = F.relu(self.cnn_extract(lstm_txt2_)).squeeze().transpose(1, 2)
        
#         cnn_encode1 = F.max_pool2d(cnn_encode1, (len(txt1[0]), 1)).squeeze()
#         cnn_encode2 = F.max_pool2d(cnn_encode2, (len(txt2[0]), 1)).squeeze()
        
        weight_matrix = torch.bmm(lstm_txt1, lstm_txt2.permute(0, 2, 1))
        weight_matrix_1 = torch.exp(weight_matrix - weight_matrix.max(2, keepdim=True)[0])
        weight_matrix_2 = torch.exp(weight_matrix - weight_matrix.max(1, keepdim=True)[0])
    
        alphas = weight_matrix_1 / weight_matrix_1.sum(2, keepdim=True)
        beta = weight_matrix_2 / weight_matrix_2.sum(1, keepdim=True)
        alphas = alphas * x1_mask[:, :, None]
        beta = beta * x2_mask[:, None, :]
        
        atten_txt1to2 = torch.bmm(alphas, lstm_txt2)
        atten_txt2to1 = torch.bmm(beta.permute(0, 2, 1), lstm_txt1)
        
        txt_substract1 = torch.abs(lstm_txt1-atten_txt1to2)
        txt_substract2 = torch.abs(lstm_txt2-atten_txt2to1)
        
        txt_multiply1 = torch.mul(lstm_txt1, atten_txt1to2)
        txt_multiply2 = torch.mul(lstm_txt2, atten_txt2to1)
        
        out_txt1 = torch.cat([lstm_txt1, atten_txt1to2, txt_substract1, txt_multiply1, cnn_encode1], 2)
        out_txt2 = torch.cat([lstm_txt2, atten_txt2to1, txt_substract2, txt_multiply2, cnn_encode2], 2)
        lstm_fusion_txt1 = self.lstm_fusion(out_txt1, None)[0]
        lstm_fusion_txt2 = self.lstm_fusion(out_txt2, None)[0]
        lstm_fusion_txt1 = lstm_fusion_txt1.unsqueeze(1)
        lstm_fusion_txt2 = lstm_fusion_txt2.unsqueeze(1)
        lstm_txt1_max = F.max_pool2d(lstm_fusion_txt1, (len(txt1[0]), 1)).squeeze()
        lstm_txt2_max = F.max_pool2d(lstm_fusion_txt2, (len(txt2[0]), 1)).squeeze() 
        output = self.linear_1(torch.cat([lstm_txt1_max, lstm_txt2_max], 1))
        if self.is_batch_norm:
            output = self.batch_norm_1(output)
        output = F.relu(output)
        if self.is_drop_out:
            output = self.linear_1_dropout(output)
        for i in range(1, len(self.deep_layers)):
            output = getattr(self, 'linear_' + str(i + 1))(output)
            if self.is_batch_norm:
                output = getattr(self, 'batch_norm_' + str(i + 1))(output)
            output = F.relu(output)
            if self.is_drop_out:
                output = getattr(self, 'linear_' + str(i + 1) + '_dropout')(output)
        output = self.deep_out(output)
        return F.log_softmax(output)

In [70]:
class Hybrid_rnn_cnn_V2(nn.Module):
    def __init__(self, vocab, embed_dim, hidden_dim, deep_layers, pretrain_embed=torch.tensor([])):
        super(Hybrid_rnn_cnn_V2, self).__init__()
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.deep_layers = deep_layers
        self.word_embed = nn.Embedding(len(vocab), embed_dim, padding_idx=vocab.stoi['<PAD>'])
        if len(pretrain_embed) != 0:
            self.word_embed.weight.data.copy_(pretrain_embed)
        self.lstm_embed = nn.LSTM(embed_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.conv_step2 = nn.Conv2d(1, 40, (hidden_dim*2, 2))
        self.conv_step3 = nn.Conv2d(1, 40, (hidden_dim*2, 3))
        self.conv_step4 = nn.Conv2d(1, 40, (hidden_dim*2, 4))
        self.lstm_fusion = nn.LSTM(2 * hidden_dim * 4, hidden_dim, batch_first=True, bidirectional=True)
        self.deep_layer_1 = nn.Linear(4*hidden_dim, deep_layers[0])
        self.deep_out = nn.Linear(deep_layers[0], 2)
#         self.deep_dropout_1 = nn.Dropout(0.5)
    def forward(self, txt1, txt2, hidden=None):
        embed_txt1 = self.word_embed(txt1)
        embed_txt2 = self.word_embed(txt2)
        lstm_txt1 = self.lstm_embed(embed_txt1, None)[0]
        lstm_txt2 = self.lstm_embed(embed_txt2, None)[0]
        
        lstm_txt1_ = lstm_txt1.transpose(2, 1).unsqueeze(1)
        lstm_txt2_ = lstm_txt2.transpose(2, 1).unsqueeze(1)
        lstm_txt1_step2 = F.relu(self.conv_step2(lstm_txt1_)).permute(0, 2, 3, 1)
        lstm_txt2_step2 = F.relu(self.conv_step2(lstm_txt2_)).permute(0, 2, 3, 1)
        lstm_txt1_step3 = F.relu(self.conv_step3(lstm_txt1_)).permute(0, 2, 3, 1)
        lstm_txt2_step3 = F.relu(self.conv_step3(lstm_txt2_)).permute(0, 2, 3, 1)
        lstm_txt1_step4 = F.relu(self.conv_step4(lstm_txt1_)).permute(0, 2, 3, 1)
        lstm_txt2_step4 = F.relu(self.conv_step4(lstm_txt2_)).permute(0, 2, 3, 1)
        
        
        lstm_txt1_step2 = F.max_pool2d(lstm_txt1_step2, (lstm_txt1_step2.size(2), 1)).squeeze()
        lstm_txt2_step2 = F.max_pool2d(lstm_txt2_step2, (lstm_txt2_step2.size(2), 1)).squeeze()
        lstm_txt1_step3 = F.max_pool2d(lstm_txt1_step3, (lstm_txt1_step3.size(2), 1)).squeeze()
        lstm_txt2_step3 = F.max_pool2d(lstm_txt2_step3, (lstm_txt2_step3.size(2), 1)).squeeze()
        lstm_txt1_step4 = F.max_pool2d(lstm_txt1_step4, (lstm_txt1_step4.size(2), 1)).squeeze()
        lstm_txt2_step4 = F.max_pool2d(lstm_txt2_step4, (lstm_txt2_step4.size(2), 1)).squeeze()
        
#         convout_txt1 = torch.cat([lstm_txt1_step2, lstm_txt1_step3, lstm_txt1_step4], 1)
#         convout_txt2 = torch.cat([lstm_txt2_step2, lstm_txt2_step3, lstm_txt2_step4], 1)
        
        txt_substract_step2 = torch.abs(lstm_txt1_step2 - lstm_txt2_step2)
        txt_multiply_step2 = torch.mul(lstm_txt1_step2, lstm_txt2_step2)
        txt_substract_step3 = torch.abs(lstm_txt1_step3 - lstm_txt2_step3)
        txt_multiply_step3 = torch.mul(lstm_txt1_step3, lstm_txt2_step3)
        txt_substract_step4 = torch.abs(lstm_txt1_step4 - lstm_txt2_step4)
        txt_multiply_step4 = torch.mul(lstm_txt1_step4, lstm_txt2_step4)
    
        lstm_txt1 = lstm_txt1.unsqueeze(1)
        lstm_txt2 = lstm_txt2.unsqueeze(1)
        lstm_txt1 = F.max_pool2d(lstm_txt1, (len(txt1[0]), 1)) 
        lstm_txt2 = F.max_pool2d(lstm_txt2, (len(txt2[0]), 1))
        lstm_txt1 = lstm_txt1.squeeze()
        lstm_txt2 = lstm_txt2.squeeze()
        txt_substract = torch.abs(lstm_txt1 - lstm_txt2)
        txt_multiply = torch.mul(lstm_txt1, lstm_txt2)
        output = F.relu(self.deep_layer_1(torch.cat([
                                                     txt_substract, txt_multiply], 1)))
        output = self.deep_out(output)
        return F.log_softmax(output)

In [71]:
model = Hybrid_rnn_cnn_V2(TEXT.vocab, EMBED_DIM, HIDDEN_DIM, DEEP_LAYERS, matrix)
model.to(DEVICE)
criterion = nn.NLLLoss(weight=torch.tensor(CLASS_WEIGHT).float().to(DEVICE))
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = StepLR(optimizer, step_size=DECAY_STEP, gamma=DECAY_GAMMA)

In [72]:
print_every = 50
best_state = None
max_metric = 0
# model.to(DEVICE)
def predict(model, data_dl, loss_func, device):
    model.eval()
    res_list = []
    label_list = []
    loss = 0
    for text1, text2, label in data_dl:
        text1 = text1.to(DEVICE)
        text2 = text2.to(DEVICE)
        label = label.to(DEVICE)
#         x1_mask, x2_mask = get_mask(text1, text2)
        y_pred = model(text1, text2)
        loss += loss_func(y_pred, label).data.cpu()
        y_pred = y_pred.data.max(1)[1].cpu().numpy()
        res_list.extend(y_pred)
        label_list.extend(label.data.cpu().numpy())
    acc = accuracy_score(res_list, label_list)
    Precision = precision_score(res_list, label_list)
    Recall = recall_score(res_list, label_list)
    F1 = f1_score(res_list, label_list)
    
    return loss, (acc, Precision, Recall, F1)
        
def evaluate(model, txt1, txt2, y):
    pred = model(txt1, txt2)
    out_batch = pred.data.max(1)[1].cpu().numpy()
    F1 = f1_score(out_batch, y)
    return F1

def training_termination(valid_result):
    if len(valid_result) >= 4:
        if valid_result[-1] < valid_result[-2] and \
            valid_result[-2] < valid_result[-3] and \
            valid_result[-3] < valid_result[-4]:
            return True
    return False

valid_iter.create_batches()
valid_batch_num = len(list(valid_iter.batches))
print_flush('start train...')
train_iter.create_batches()
batch_num = len(list(train_iter.batches))
print_flush('batch number %d '%batch_num)
valid_result = []
for epoch in range(EPOCHES):
    for param_group in optimizer.param_groups:
        print('learning rate: %.6f'% param_group['lr'])
    epoch_begin = time()
    total_loss = 0.0
    train_iter.init_epoch()
    batch_count = 0
    batch_begin_time = time()
    for text1, text2, label in train_dl:
        model.train()
        text1 = text1.to(DEVICE)
        text2 = text2.to(DEVICE)
        label = label.to(DEVICE)
#         x1_mask, x2_mask = get_mask(text1, text2)
        y_pred = model(text1, text2)
        loss = criterion(y_pred, label)
        model.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        batch_count += 1
        if batch_count % print_every == 0:
            metric = evaluate(model.eval(), text1, text2, label)
            print_flush('[%d %d] loss: %.6f metric: %.6f time: %.1f s' %
                  (epoch + 1, batch_count, total_loss / print_every, metric, time() - batch_begin_time))
            total_loss = 0.0
            batch_begin_time = time()
#     scheduler.step()
    print_flush("Evaluating....")
    loss, (acc, Precision, Recall, F1) = predict(model, valid_dl, criterion, DEVICE)
    valid_result.append(F1)
    print_flush('*'*60)
    print_flush('[epoch %d]. loss: %.6f acc: %.6f f1: %.6f time: %.1f s'%(epoch+1, loss/valid_batch_num, acc, F1, time()-epoch_begin))
    print_flush('*'*60)
    if F1 > max_metric:
        best_state = model.state_dict()
        max_metric = F1
        print_flush("save model...")
        torch.save(best_state, '../datasets/models/baseline_LSTM.pth')
    epoch_begin = time()
    if training_termination(valid_result):
        print_flush("early stop at [%d] epoch!" % (epoch+1))
        break

start train...
batch number 325 
learning rate: 0.000950


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:62: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


[1 50] loss: 0.605378 metric: 0.481481 time: 1.9 s
[1 100] loss: 0.582363 metric: 0.546875 time: 1.8 s
[1 150] loss: 0.534018 metric: 0.635135 time: 1.9 s
[1 200] loss: 0.520522 metric: 0.452555 time: 1.8 s
[1 250] loss: 0.530053 metric: 0.544218 time: 1.9 s
[1 300] loss: 0.507771 metric: 0.594203 time: 1.8 s
Evaluating....
************************************************************
[epoch 1]. loss: 0.494535 acc: 0.750809 f1: 0.527280 time: 13.6 s
************************************************************
save model...
learning rate: 0.000950
[2 50] loss: 0.459220 metric: 0.569697 time: 2.1 s
[2 100] loss: 0.463340 metric: 0.532258 time: 1.8 s
[2 150] loss: 0.471475 metric: 0.625000 time: 1.9 s
[2 200] loss: 0.476645 metric: 0.647059 time: 1.8 s
[2 250] loss: 0.461979 metric: 0.589928 time: 2.0 s
[2 300] loss: 0.463386 metric: 0.607407 time: 1.9 s
Evaluating....
************************************************************
[epoch 2]. loss: 0.488344 acc: 0.792050 f1: 0.555592 time: 13

KeyboardInterrupt: 

In [ ]:
lstm = nn.LSTM(5, 5, batch_first=True, bidirectional=True)

In [84]:
inputs = torch.randn(1, 2, 5)
out = lstm(inputs)